# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **

This is a classification problem because the results we are trying to predict are discrete. We have to predict whether a student requires intervention given his profile (YES,NO value). Classification returns a function that returns a number between 0 and 1. If the output returned given an input is greater than or equal to the threshold, it returns 1. It returns 0 otherwise.

Regression on the other hand returns a function that, given an input, it returns a continuous output value. For example, in the Boston housing problem, the predicted housing price can potentially range from 0 to infinity.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [4]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [5]:
# TODO: Calculate number of students
n_students = len(student_data.index)

# TODO: Calculate number of features
n_features = len(student_data.columns) - 1

# TODO: Calculate passing students
n_passed = len(student_data[student_data['passed'] == 'yes'])

# TODO: Calculate failing students
n_failed = n_students - n_passed

# TODO: Calculate graduation rate
grad_rate = n_passed / float(n_students) * 100

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [6]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [7]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        print col_data
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

0      GP
1      GP
2      GP
3      GP
4      GP
5      GP
6      GP
7      GP
8      GP
9      GP
10     GP
11     GP
12     GP
13     GP
14     GP
15     GP
16     GP
17     GP
18     GP
19     GP
20     GP
21     GP
22     GP
23     GP
24     GP
25     GP
26     GP
27     GP
28     GP
29     GP
       ..
365    MS
366    MS
367    MS
368    MS
369    MS
370    MS
371    MS
372    MS
373    MS
374    MS
375    MS
376    MS
377    MS
378    MS
379    MS
380    MS
381    MS
382    MS
383    MS
384    MS
385    MS
386    MS
387    MS
388    MS
389    MS
390    MS
391    MS
392    MS
393    MS
394    MS
Name: school, dtype: object
0      F
1      F
2      F
3      F
4      F
5      M
6      M
7      F
8      M
9      M
10     F
11     F
12     M
13     M
14     M
15     F
16     F
17     F
18     M
19     M
20     M
21     M
22     M
23     M
24     F
25     F
26     M
27     M
28     M
29     M
      ..
365    M
366    M
367    F
368    F
369    F
370    F
371    M
372    F
373    F
37

### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [8]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, train_size=num_train, random_state=42)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. What are the general applications of each model? What are their strengths and weaknesses? Given what you know about the data, why did you choose these models to be applied?*

**Answer: **

1. *** Decision Tree ***

    A decision tree uses simple rules to separate data into different groups on each level and eventually reach the leaf nodes which are the learned outputs. To determine which feature to split against, we split against each feature and use the feature that gives us the most information gain. An information gain is the reduction of entropy (randomness) and is calculated by the difference of the entropy of the parent node (before splitting) and the weighted sum of entropies of the left and right child nodes (after splitting).

    Strengths of decision trees include:

        i. It is easy to understand and interpret the results because it is simply a series of if-else statements joined together.
        ii. It is easy to visualize a decision tree and follow the chains of logic statements.
        iii. It can handle both numerical and categorical values. When given numerical values, a (C4.5) decision tree can partition numerical values into discrete intervals.
        iv. Decision trees are not sensitive to outliers because the algorithm finds the split that produces the greatest information gain, while splitting against an outlier will produce almost no information gain.

    Weaknesses of decision trees include:

        i. A decision tree that is too deep will overfit the training set and fail to produce good predictions. This problem is usually mitigated by pruning (which sklearn doesn't support currently), setting the minumum number of samples required at the leaf node and setting the maximum tree depth.
        ii. Decision trees are sensitive to small change in a dataset and produce a very different decision tree than the one before the change.
        iii. It requires redrawing the decision tree every time new data is produced. It is not possible to simply update a subset of the tree, making the execution time longer than other algorithms.

    I chose decision tree because I think there are many variables that affect whether a student will pass and the variables are strongly correlated. For example, if a student has a healthy family whose members are educated and have good jobs, the parents are more likely to want their child to receive better education by choosing a better school, giving him more educational resources and making sure he's spending enough time studying. On the other hand, a child who is raised by single parent without much education and disposable income is more likely to have less choices in terms of the education he receives, and his parent is more likely to be busy working odd jobs to support the family and not have time to make sure the child spends enough time studying. It makes sense to have a flow chart that demonstrates the chain of logic statements.

2. *** Naive Bayes Classifier ***

    A naive Bayes classifier applies Bayes' theorem to the dataset with the naive assumption that each pair of features are independent. It returns the output that has the highest probability given the input. The probability is calculated by combining the probabilities of each feature without considering the correlations among the features. Let's say we are using `Medu`, `Mjob`, `Fedu` and `Fjob` as features. The probabilities of `passed` being `yes` or `no` given each of them are calculated separately and combined together without considering the correlations. It's likely that a person's job is correlated to his education and it's also likely that a couple has similar education background (e.g. they met in college).
    
    Strengths of naive Bayes classifiers include:
    
        i. Naive Bayes classifier works surprisingly well in real-world situations even though its assumption is flawed. One of the reasons is the dependencies cancel out each other.
        ii. Naive Bayes classifier requires only a small number of data to estimate the outputs
        iii. Naive Bayes classifier is fast to train
        iv. Naive Bayes classifier is not sensitive to irrelevant features and outliers
        
    Weaknesses of naive Bayes classifiers include:
    
        i. The probabilities returned are not accurate because it is calculated based on the assumption that each pair of features are independent. In reality, it's almost certain that they are somehow correlated. When using a naive Bayes classifier, make sure that we only use the probabilities' order but not the probabilities themselves.
        
    I chose naive Bayes classifier because each of the features in the dataset can contribute independently to the final probability of whether a student will pass or fail the exam. Even though it's clear that naive Bayes classifier's `naive` approach is not realistic, it has been proven to work well in real-world scenarios and I think it will output relatively accurate predictions.
        
3. *** Support Vector Machine ***

    A support vector machine trains a function that best separates data of different classes with a hyperplane whose gap between the data points of each class is as wide as possible.
    
    Strengths of SVM include:
        
        i. Flexible choice of kernel functions with the ability to divide data non-linearly.
        ii. SVM maximizes the margin, making it more robust than other ways of separating data of different classes.
        iii. SVM is a convex optimization problem, meaning that it has no local optima.
        
    Weaknesses of SVM include:    
    
        i. SVM is slow if we want to train a dataset with multiple classes because it can only do 1 vs all, meaning that we will have to train multiple 1 vs all to determine which class a data point falls into. (It doesn't apply to our case, however)
        ii. SVM falls into the trap of the curse of dimensionality. If we have a lot more features than samples, the prediction will not be accurate unless we feed a lot more data to the model.
        
    I chose SVM because I believe the two groups of students can be separated by a clear gap without much ambiguoity (with a few exceptions). It is unlikely for a student to randomly fail an exam and it usually has to do with the student's circumstances, which I believe can be modeled and separated by SVM.

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [9]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [17]:
# TODO: Import the three supervised learning models from sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

# TODO: Initialize the three models
clf_A = RandomForestClassifier(random_state=42)
clf_B = GaussianNB()
clf_C = SVC(random_state=42)

for clf in [clf_A, clf_B, clf_C]:
    for size in [100, 200, 300]:
        train_predict(clf, X_train[:size], y_train[:size], X_test, y_test)
        print "\n"
        print "***********************************"
        print "\n"

Training a RandomForestClassifier using a training set size of 100. . .
Trained model in 0.0503 seconds
Made predictions in 0.0023 seconds.
F1 score for training set: 0.9844.
Made predictions in 0.0013 seconds.
F1 score for test set: 0.7153.


***********************************


Training a RandomForestClassifier using a training set size of 200. . .
Trained model in 0.0399 seconds
Made predictions in 0.0025 seconds.
F1 score for training set: 0.9964.
Made predictions in 0.0011 seconds.
F1 score for test set: 0.7857.


***********************************


Training a RandomForestClassifier using a training set size of 300. . .
Trained model in 0.0336 seconds
Made predictions in 0.0015 seconds.
F1 score for training set: 0.9951.
Made predictions in 0.0011 seconds.
F1 score for test set: 0.8092.


***********************************


Training a GaussianNB using a training set size of 100. . .
Trained model in 0.0007 seconds
Made predictions in 0.0003 seconds.
F1 score for training set:

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - Random Forest Classifier **  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100 | 0.0503 | 0.0023 | 0.9844 | 0.7153 |
| 200 | 0.0339 | 0.0025 | 0.9964 | 0.7857 |
| 300 | 0.0336 | 0.0015 | 0.9951 | 0.8092 |

** Classifer 2 - Naive Bayes Gaussian Classifier **  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100 | 0.0007 | 0.0003 | 0.8467 | 0.8029 |
| 200 | 0.0009 | 0.0004 | 0.8406 | 0.7244 |
| 300 | 0.0010 | 0.0005 | 0.8038 | 0.7634 |

** Classifer 3 - Support Vector Machine **  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100 | 0.0016 | 0.0010 | 0.8777 | 0.7746 |
| 200 | 0.0042 | 0.0039 | 0.8679 | 0.7815 |
| 300 | 0.0073 | 0.0058 | 0.8761 | 0.7838 |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Chosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **

I think support vector machine is the most appropriate model because

1. although it's not the most accurate model as the training set grows, its accuracy is very close to the most accurate model `RandomForestClassifier`.
2. its training time is significantly lower than `RandomForestClassifier` while its prediction times are not much higher than `RandomForestClassifier`. Given that potentially we can have more and more student profiles, training time is an important factor to consider. A decision tree has a disadvantage of having to rebuild the model every time new student profiles come in and the gaps in training time between itself and the other classifiers will widen.
. 

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. For example if you've chosen to use a decision tree or a support vector machine, how does the model go about making a prediction?*

**Answer: **

A support vector machine draws a line between the students who passed the exam and the students who failed the exam. This line maximizes the gap between the students who passed and the students who failed. We are able to make complex decisions because the kernel is able to separate students even with 30 different characteristics. When we have a new student profile, we predict whether a student will pass the exam by looking at which decision boundary the student falls into.

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [16]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.cross_validation import StratifiedShuffleSplit

# TODO: Create the parameters list you wish to tune
parameters = {
    'C': [0.1, 1, 10, 100, 1000],        
    'kernel': ['rbf', 'sigmoid'],
    'gamma': [1e-3, 1e-4, 1e-5]
}

cv = StratifiedShuffleSplit(y_train, n_iter=3, test_size=0.3)
    
# TODO: Initialize the classifier
clf = SVC(random_state=42)

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label="yes")

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, param_grid=parameters, cv=cv, scoring=f1_scorer)


# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

Made predictions in 0.0046 seconds.
Tuned model has a training F1 score of 0.8323.
Made predictions in 0.0016 seconds.
Tuned model has a testing F1 score of 0.7945.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **

The final model's F<sub>1</sub> score for training and testing are 0.8323 and 0.7945. The test score is slightly better than the untuned model.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.